'''
@Author: Naziya

@Date: 2021-10--05

@Last Modified by: Naziya

@Last Modified : 2021-10-05

@Title : Program Aim is to clean and preprocess the data recieved from kafka and then create a dataframe and build a machine learning model. 
'''


In [1]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Stock Data processing').getOrCreate()

21/10/07 02:45:12 WARN Utils: Your hostname, naziya-Inspiron-N5110 resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface wlp9s0)
21/10/07 02:45:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/07 02:45:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/07 02:45:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import *

Read the data

In [3]:
df = spark.read.csv("hdfs://localhost:9000/stock_price_data/stock_data.csv",header=True)
df.head()

Row(["time"='["2021-10-04 20:00:00"',  "open"=' "144.12"',  "high"=' "144.12"',  "low"=' "144.12"',  "close"=' "144.12"',  "volume"]=' "500"]')

preprocessing the data

In [4]:
dataset2=df.withColumnRenamed('["time"','time')\
    .withColumnRenamed(' "open"','open')\
    .withColumnRenamed(' "high"','high')\
    .withColumnRenamed(' "low"','low')\
    .withColumnRenamed(' "close"','close')\
    .withColumnRenamed(' "volume"]','volume')

In [5]:
dataset2.describe()

DataFrame[summary: string, time: string, open: string, high: string, low: string, close: string, volume: string]

In [6]:
dataset2.show(4)

+--------------------+---------+---------+---------+---------+--------+
|                time|     open|     high|      low|    close|  volume|
+--------------------+---------+---------+---------+---------+--------+
|["2021-10-04 20:0...| "144.12"| "144.12"| "144.12"| "144.12"|  "500"]|
|["2021-10-04 19:4...| "144.27"| "144.27"| "144.27"| "144.27"|  "200"]|
|["2021-10-04 18:3...| "144.44"| "144.44"| "144.44"| "144.44"|  "101"]|
|["2021-10-04 18:2...|  "144.4"| "144.44"| "144.35"| "144.44"| "2895"]|
+--------------------+---------+---------+---------+---------+--------+
only showing top 4 rows



Data cleaning

In [8]:
new_df = dataset2.withColumn('open', regexp_replace('open', '"', ''))\
    .withColumn('Time', regexp_replace('time', '\\["', ''))\
    .withColumn('Time', regexp_replace('time', '"', ''))\
    .withColumn('High', regexp_replace('high', '"', ''))\
    .withColumn('Low', regexp_replace('low', '"', ''))\
    .withColumn('Close', regexp_replace('close', '"', ''))\
    .withColumn('Volume', regexp_replace('volume', '"', ''))\
    .withColumn('Volume', regexp_replace('volume', '\\]', ''))


Casting the string values to double

In [9]:
df2 = new_df\
    .withColumn("open",col("Open").cast("double"))\
    .withColumn("high",col("High").cast("double"))\
    .withColumn("low",col("Low").cast("double"))\
    .withColumn("close",col("Close").cast("double"))\
    .withColumn("volume",col("Volume").cast("double"))

In [10]:
df2.show(4)

+-------------------+------+------+------+------+------+
|               Time|  open|  high|   low| close|volume|
+-------------------+------+------+------+------+------+
|2021-10-04 20:00:00|144.12|144.12|144.12|144.12| 500.0|
|2021-10-04 19:45:00|144.27|144.27|144.27|144.27| 200.0|
|2021-10-04 18:30:00|144.44|144.44|144.44|144.44| 101.0|
|2021-10-04 18:25:00| 144.4|144.44|144.35|144.44|2895.0|
+-------------------+------+------+------+------+------+
only showing top 4 rows



Creating the vectors from features. Apache MLib takes the input in the form of vectors

In [11]:
featureassembler=VectorAssembler(inputCols=["open","high","low"],outputCol="features")
output=featureassembler.transform(df2)
output.select("features").show(5,truncate=False)

+----------------------+
|features              |
+----------------------+
|[144.12,144.12,144.12]|
|[144.27,144.27,144.27]|
|[144.44,144.44,144.44]|
|[144.4,144.44,144.35] |
|[144.35,144.45,144.3] |
+----------------------+
only showing top 5 rows



Final data with the features and label(close)

In [12]:
finalized_data=output.select("time","features","close").sort("time",ascending=True)
finalized_data.show(5)


+-------------------+--------------------+------+
|               time|            features| close|
+-------------------+--------------------+------+
|2021-09-07 11:50:00|[138.4793,138.49,...|138.44|
|2021-09-07 11:55:00|[138.45,138.4567,...|138.19|
|2021-09-07 12:00:00|[138.18,138.32,13...|138.21|
|2021-09-07 12:05:00|[138.21,138.44,13...|138.44|
|2021-09-07 12:10:00|[138.46,138.545,1...|138.51|
+-------------------+--------------------+------+
only showing top 5 rows



Splitting data into training and testing data

In [13]:
final_data = finalized_data.withColumn("rank",percent_rank().over(Window.partitionBy().orderBy("time")))

In [14]:
train_data = final_data.where("rank <= .8").drop("rank")
test_data = final_data.where("rank > .8").drop("rank")

In [15]:
train_data.count()

21/10/07 02:57:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


1512

In [16]:
test_data.count()

21/10/07 02:57:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


378

In [17]:
type(test_data)

pyspark.sql.dataframe.DataFrame

In [19]:
test_data.write.parquet('testdata1')

21/10/07 02:58:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Building a ML Model (Linear regression)

In [20]:
linear_reg=LinearRegression(featuresCol='features',labelCol='close')
lr_model=linear_reg.fit(train_data)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

21/10/07 02:59:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/10/07 02:59:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/10/07 02:59:29 WARN Instrumentation: [c3b6bd60] regParam is zero, which might cause numerical instability and overfitting.
21/10/07 02:59:31 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/10/07 02:59:31 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/10/07 02:59:31 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/10/07 02:59:31 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
21/10/07 02:59:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partitio

Coefficients: [-0.47754351773623854,0.7533940437096085,0.724553223959886]
Intercept: -0.06180391786587447


In [21]:
predDF = lr_model.transform(test_data)
predDF.select("features", "close","prediction").show(5)

21/10/07 03:00:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+------+------------------+
|            features| close|        prediction|
+--------------------+------+------------------+
|[139.76,139.76,13...|139.58|139.61695906025358|
|[139.56,139.58,13...|139.33|139.38847299770353|
|[139.33,139.46,13...|139.46|139.40579951718823|
|[139.46,139.48,13...|139.38|139.39588386582344|
|[139.36,139.3865,...|139.33|  139.283858461996|
+--------------------+------+------------------+
only showing top 5 rows



Evaluating the models

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
regressionEvaluator = RegressionEvaluator(
predictionCol="prediction",
labelCol="close",
metricName="rmse")
rmse = regressionEvaluator.evaluate(predDF)
print(f"RMSE is {rmse:.1f}")


21/10/07 03:00:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


RMSE is 0.1


Save model

In [27]:
lr_model.save("hdfs://localhost:9000/stock_price_data_model")